In [412]:
import numpy as np

In [413]:
def keyMatrix(key):
    if(len(key) != 9):
        print("Key should be of length 9")
        return np.array([1,0,0],[0,1,0],[0,0,1])
    for i in range(9):
        if(key[i]<'A' or key[i]>'Z'):
            print("Key should be from A to Z")
            return np.array([1,0,0],[0,1,0],[0,0,1])
    keyMatrix = np.zeros((3,3),dtype=int)
    for i in range(3):
        for j in range(3):
            keyMatrix[i][j] = (ord(key[3*i+j])-ord('A'))%26
    return keyMatrix

In [414]:
#testing the KeyMatrix function
print(keyMatrix("GYBNQKURP"))

[[ 6 24  1]
 [13 16 10]
 [20 17 15]]


In [415]:
def encrypt(message,key):
    message = message.upper()
    key = key.upper()
    
    keyMat = keyMatrix(key)
    n = len(message)
    if(n%3!=0):
        message += 'X'*(3-n%3)
        n = len(message)
    cipher = ""
    for i in range(0,n,3):
        messageVec = np.zeros((3,1),dtype=int)
        for j in range(3):
            messageVec[j][0] = ord(message[i+j])-ord('A')
        cipherVec = np.dot(keyMat,messageVec)%26
        for j in range(3):
            cipher += chr(cipherVec[j][0]+ord('A'))
    return cipher

In [416]:
#testing the encrypt function
print(encrypt("ACT","GYBNQKURP"))
print(encrypt("THISISYOU","GYBNQKURP"))

POH
EXVGWMGIE


In [417]:
def AdotBinv(A,B):
    # A dot B inverse using modulo function so that it can be used for Hill Cipher and no float terms
    adjoint = np.zeros((3,3),dtype=int)
    for i in range(3):
        for j in range(3):
            adjoint[i][j] = (B[(j+1)%3][(i+1)%3]*B[(j+2)%3][(i+2)%3] - B[(j+1)%3][(i+2)%3]*B[(j+2)%3][(i+1)%3])
    # print(adjoint)
    det_B = (B[0][0]*(B[1][1]*B[2][2]-B[1][2]*B[2][1]) - B[0][1]*(B[1][0]*B[2][2]-B[1][2]*B[2][0]) + B[0][2]*(B[1][0]*B[2][1]-B[1][1]*B[2][0]))
    # print(det_B)
    det_inv = pow(int(det_B),-1,26)
    ans = np.matmul(A,adjoint)%26
    ans = (ans*det_inv)%26
    return ans
    

KeyDiscovery Concept is that
cipher_text = C
key = K
msg_text = M

C = K*M mod 26
K = C*M_inv mod 26
(All operations are also modulo 26)


In [418]:
def key_discovery(cipher,message):
    message = message.upper()
    cipher = cipher.upper()
    
    #given text ABCDEFGHI matrix looks like this [[0,3,6],[1,4,7],[2,5,8]]
    if(len(message) < 9):
        print("Key can't be resolved")
        return "AAAAAAAAA"
    msgMat = np.zeros((3,3),dtype=int)
    for i in range(3):
        for j in range(3):
            msgMat[j][i] = ord(message[3*i+j])-ord('A')
    det = (msgMat[0][0]*(msgMat[1][1]*msgMat[2][2]-msgMat[1][2]*msgMat[2][1]) - msgMat[0][1]*(msgMat[1][0]*msgMat[2][2]-msgMat[1][2]*msgMat[2][0]) + msgMat[0][2]*(msgMat[1][0]*msgMat[2][1]-msgMat[1][1]*msgMat[2][0]))
    
    #checking if key can be resolved
    #only if mod_inv of det w.r.t 26 exists
    gcd = np.gcd(det,26)
    if(gcd!=1):
        return key_discovery(cipher[3:],message[3:])
    
    cipherMat = np.zeros((3,3),dtype=int)
    for i in range(3):
        for j in range(3):
            cipherMat[j][i] = ord(cipher[3*i+j])-ord('A')
            
    keyMat = AdotBinv(cipherMat,msgMat)
    key = ""
    for i in range(9):
        key += chr(keyMat[i//3][i%3]+ord('A'))
    return key

In [419]:
#testing the key_discovery function
msgtext = "WHATAREYOUDOINGTHESEDAYSIAMWORKINGONHILLCIPHER"
key = "HILLISEZY"

# calculating the cipher text
cipher = encrypt(msgtext,key)
print("cipher",cipher)

discovered_key = key_discovery(cipher,msgtext)
print("discovered_key",discovered_key)

#again checking the key
cipher_checked = encrypt(msgtext,discovered_key)
print("cipher_checked",cipher_checked)


if(cipher == cipher_checked and key == discovered_key):
    print("Key found successfully")

cipher CMDIVQKUQGCXSOHZZJJYKAWSGSILKORSGLWKABYZVAXHTKFZ
discovered_key HILLISEZY
cipher_checked CMDIVQKUQGCXSOHZZJJYKAWSGSILKORSGLWKABYZVAXHTKFZ
Key found successfully
